In [47]:
import pandas as pd
import numpy as np
# Video de guia https://www.youtube.com/watch?v=djj5RMN7yLw&t=464s

In [48]:
items = pd.read_csv("Csv_limpios/steam_items_limpio.csv")
reviews = pd.read_csv("Csv_limpios/steam_reviews_limpio.csv")
steam_games = pd.read_csv("Csv_limpios/steam_games_limpio.csv")
steam_games = steam_games.head(1000)
steam_games

,genres,title,price,id,año
0,Action,Lost Summoner Kitty,4.99,761140,2018
1,Casual,Lost Summoner Kitty,4.99,761140,2018
2,Indie,Lost Summoner Kitty,4.99,761140,2018
3,Simulation,Lost Summoner Kitty,4.99,761140,2018
4,Strategy,Lost Summoner Kitty,4.99,761140,2018
...,...,...,...,...,...
995,Adventure,Hamilton's Great Adventure,9.99,42140,2011
996,Casual,Hamilton's Great Adventure,9.99,42140,2011
997,Indie,Hamilton's Great Adventure,9.99,42140,2011
998,Simulation,The Sims™ 3 Generations,19.99,47898,2011


In [59]:
#Importamos TfidfVectorizer para pasar las palabras a número
steam_games = pd.read_csv("Csv_limpios/steam_games_limpio.csv")
steam_games = steam_games.head(1000)

from sklearn.feature_extraction.text import TfidfVectorizer

#Le asignamos el Vectorizer a una variable
tfidf = TfidfVectorizer()

#Llenamos los datos nulos con un espacio en blanco
steam_games["genres"] = steam_games["genres"].fillna(" ")

#Entrenamos el modelo
tfidf_matrix = tfidf.fit_transform(steam_games["genres"])

#Comparamos las etiquetas para seleccionar la que mayor similitud tiene
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

steam_games.set_index("title", inplace=True)

def recomendacion_juego(id):
    # Aseguramos que el título esté presente en el DataFrame
    #if id not in steam_games["id"]:
    #    return f"El juego con la ID '{id}' no se encuentra en la base de datos."

    # Obtenemos el índice del juego
    idx = steam_games.index.get_loc(id)

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    
    juegos_indices = [i[0] for i in sim_scores]
    return steam_games.iloc[juegos_indices].index  # Devolvemos los títulos recomendados

# Ejemplo de uso con la ID del juego
juego_a_recomendar_id = 761140  # Reemplaza 'id_del_juego' con la ID real
recomendaciones = recomendacion_juego(juego_a_recomendar_id)

print(f"Recomendaciones para el juego con ID '{juego_a_recomendar_id}':")
print(recomendaciones)

KeyError: 761140

In [50]:
recomendacion_juego(761140)

"El juego con la ID '761140' no se encuentra en la base de datos."

In [51]:
steam_games.columns

Index(['genres', 'price', 'id', 'año'], dtype='object')

In [58]:
steam_games.dtypes

genres     object
price     float64
id          int64
año         int64
dtype: object